In [1]:
import matplotlib
%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2

import nest_asyncio
%config Completer.use_jedi = False

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 
import numpy as np
import os
import scipy

# add root directory to be able to import packages
# todo: make all packages installable so they can be called/imported by environment
import sys
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

from utils.calcium import calcium


Autosaving every 180 seconds


/home/cat/code/donatolab/manifolds/utils/animal_database/animal_database.py:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
############################################
################# Load data ################
############################################

#
def get_sessions_from_directory_listing(root_dir,
                                       animal_id):
    
    sessions = os.listdir(root_dir+animal_id)
    print ("sessions: ", sessions)
    
    return sessions
    
#
root_dir = '/media/cat/4TB1/donato/'
animal_id = 'DON-009192'
dir_ = '/002P-F/tif/'

sessions = get_sessions_from_directory_listing(root_dir,
                                              animal_id)

#
for session in sessions:
    #
    try: 
        print ("processing: ", session)
        c = calcium.Calcium()
        c.root_dir = root_dir
        c.data_dir = os.path.join(root_dir, animal_id, session+dir_, 'suite2p','plane0')
        c.animal_id = animal_id
        c.session = session
        c.detrend_model_order = 1
        c.load_suite2p()
        c.save_python = True
        c.save_matlab = False

        # 
        c.load_binarization()
        binarization_method = 'upphase'
        if binarization_method=='onphase':
            traces = c.F_onphase_bin
        elif binarization_method=='upphase':
            traces = c.F_upphase_bin
        else:
            print ("METHOD NOT FOUND")

        #print ("binarized data: ", traces.shape)

        ###################################################################
        ########## cleanup cells + compute pairwise correlations ##########
        ###################################################################
        c.load_footprints()
        c.deduplication_method = 'overlap'      # 'overlap'; 'centre_distance'
        c.corr_min_distance = 8                 # min distance for centre_distance method - NOT USED HERE
        c.corr_max_percent_overlap = 0.25       # max overlap for overlap method
        c.corr_threshold = 0.3                  # max correlation allowed for high overlap

        #  
        c.corr_delete_method = 'lowest_snr' #'highest_connected', lowest_snr'
        c.recompute_deduplication = False

        c.remove_duplicate_neurons()
    except:
        print ("... errror loading : ", session)


sessions:  ['20220317', '20220323', '20220325', '20220219', '20220314', '20220315', '20220316', '20220313', '20220224', '20220307', '20220330', '20220223', '20220227', '20220305', '20220304', '20220311', '20220222', '20220321', '20220310', '20220322', '20220324', '20220216', '20220226', '20220320', '20220217', '20220303', '20220308', '20220220', '20220221', '20220328', '20220302', '20220215', '20220225', '20220329', '20220319', '20220309', '20220326', '20220327', '20220312', '20220318', '20220401', '20220306', '20220228', '20220218', '20220331']
processing:  20220317
check #1: 
check #2: 
check #3: 
processing:  20220323
... errror loading :  20220323
processing:  20220325
check #1: 
check #2: 
check #3: 
processing:  20220219
check #1: 
check #2: 
check #3: 
processing:  20220314
check #1: 
check #2: 
check #3: 
processing:  20220315
check #1: 
check #2: 
check #3: 
processing:  20220316
check #1: 
check #2: 
check #3: 
processing:  20220313
check #1: 
check #2: 
check #3: 
processing

/home/cat/code/donatolab/manifolds/utils/calcium/calcium.py:251: RuntimeWarning: invalid value encountered in true_divide
  cumsum = cumsum/np.max(cumsum)


... errror loading :  20220302
processing:  20220215
check #1: 
check #2: 
check #3: 
processing:  20220225
check #1: 
check #2: 
check #3: 
processing:  20220329
... errror loading :  20220329
processing:  20220319
check #1: 
check #2: 
check #3: 
processing:  20220309
check #1: 
check #2: 
check #3: 
processing:  20220326


/home/cat/code/donatolab/manifolds/utils/calcium/calcium.py:251: RuntimeWarning: invalid value encountered in true_divide
  cumsum = cumsum/np.max(cumsum)


... errror loading :  20220326
processing:  20220327
... errror loading :  20220327
processing:  20220312
check #1: 
check #2: 
check #3: 
processing:  20220318
check #1: 
check #2: 
check #3: 
processing:  20220401
... errror loading :  20220401
processing:  20220306
check #1: 
check #2: 
check #3: 
processing:  20220228
check #1: 
check #2: 
check #3: 
processing:  20220218
check #1: 
check #2: 
check #3: 
processing:  20220331
... errror loading :  20220331


In [7]:
def get_sessions_from_filename(fname):
    
    fname = fname.replace("-","_")
    
    idx1 = fname.find("002P_F")+7
    idx2 = fname.find('.mesc')
    
    sessions = fname[idx1:idx2].replace("-","_").split("_")
    
    try:
        sessions.remove("ACQ")
    except:
        pass
    
    return sessions


def get_sessions_from_filename2(fname):

    sessions = []
    for k in range(1,20,1):
        temp = "S"+str(k)
        if temp in fname:
            sessions.append(temp)

    return sessions



def get_sessions_for_mesc_and_raw_filenames_from_directory(folder):
    
    fs = []
    ss = []
    
    # look for .mesc files
    extensions = '.mesc'
    
    matches = []
    for root, dirnames, filenames in os.walk(folder):
        #print (root, dirnames, filenames)
        for filename in filenames:
            if filename.endswith(extensions):
                matches.append(os.path.join(root, filename))
                
    for match in matches:
        sessions = get_sessions_from_filename2(match)
        fs.append(match)
        ss.append(sessions)
        #print (match, sessions)
        
    # look for .raw files
    extensions = '.raw'
    matches = []
    for root, dirnames, filenames in os.walk(folder):
        for filename in filenames:
            if filename.endswith(extensions):
                matches.append(os.path.join(root, filename))
    
    for match in matches:
        fs.append(match)
        ss.append([])           

        
    return fs, ss

folder = '/media/cat/4TB1/donato/tests'
fnames, sessions = get_sessions_for_mesc_and_raw_filenames_from_directory(folder)
for k in range(len(fnames)):
    print (fnames[k], sessions[k])

/media/cat/4TB1/donato/tests/DON-010473_20220514_002P-F_S1-S2-ACQ.mesc ['S1', 'S2']


In [9]:
def convert_mesc_sessions_to_concatenated_tiff(fname,
                                               sessions_in):

    fname_out = fname.replace('.mesc','.tif')

    if os.path.exists(fname_out):
        print (".mesc -> .tiff file already done... skipping conversion...")
        return fname_out

    # sessions_in = sessions_in.replace(" ", "").split (",")

    #
    sess_list = []
    for session in sessions_in:
        temp = session.replace("S",'')
        temp = 'MUnit_'+str(int(temp)-1)
        print ("session loaded: ", temp)
        sess_list.append(temp)
    #
    
    

#fname = fnames[0]
#sessions_in = sessions[0]

convert_mesc_sessions_to_concatenated_tiff(fname, sessions_in)




session loaded:  MUnit_0
session loaded:  MUnit_1


In [ ]:
# TEST CONTENT OF .mesc files
import h5py
def get_data(fname,
             unit_ids):

    data = []
    with h5py.File(fname, 'r') as file:

        sessions_in = file.keys()
        print ("sessions; ", sessions_in)
                    #
        for sess in sessions_in:
            print ("processing: ", sess)

            units = file[sess].keys()
            
            munits = []
            for unit in units:
                munits.append(unit)
                
            print ("Munits: ", munits)
            
            for unit_id in unit_ids:
                
                temp = file['MSession_0'][munits[unit_id]]['Channel_0'][()]
                print ("    data loaded size: ", temp.shape)
                data.append(temp)
                
    data = np.vstack(data)
    print(data.shape)


fname = '/media/cat/4TB1/donato/DON-009192/DON-009192_20220401_002P-F_S1-S2.mesc'
#fname = '/media/cat/4TB1/donato/DON-009192/DON-010473_20220510_002P-F_S1-ACQ.mesc'
#fname = '/media/cat/4TB1/donato/DON-009192/DON-009192_20220322_002P-F_S1-S2-S3-S4.mesc'
fname = '/media/cat/4TB1/donato/DON-009192/DON-010477_20220511_002P-F_S1-S2-ACQ.mesc'
fname = '/media/cat/4TB1/donato/DON-009192/DON-009191_20220318_002P-F_S1.mesc'
fname = '/media/cat/4TB1/donato/DON-009192/DON-009191_20220311_002P-F_S1-S2.mesc'
fname = '/media/cat/4TB1/donato/DON-009192/DON-009192_20220322_002P-F_S1-S2-S3-S4.mesc'


print ("fname: ", fname)
unit_ids = [0,1,2,3]
get_data(fname, unit_ids)





fname:  /media/cat/4TB1/donato/DON-009192/DON-009192_20220322_002P-F_S1-S2-S3-S4.mesc
sessions;  <KeysViewHDF5 ['MSession_0']>
processing:  MSession_0
Munits:  ['MUnit_0', 'MUnit_1', 'MUnit_2', 'MUnit_3', 'MUnit_4']
    data loaded size:  (27874, 512, 512)
    data loaded size:  (27874, 512, 512)
    data loaded size:  (27874, 512, 512)
    data loaded size:  (27874, 512, 512)


In [45]:
fname = '/media/cat/4TB1/donato/DON-009192/DON-010473_20220510_002P-F_S1-ACQ.mesc'
print (fname)
get_data(fname)



/media/cat/4TB1/donato/DON-009192/DON-010473_20220510_002P-F_S1-ACQ.mesc
sessions;  <KeysViewHDF5 ['MSession_0']>
processing:  MSession_0
<KeysViewHDF5 ['MUnit_0']>


In [ ]:
def convert_mesc_sessions_to_concatenated_tiff(fname,
                                               sessions_in):

    fname_out = fname.replace('.mesc','.tif')

    if os.path.exists(fname_out):
        print (".mesc -> .tiff file already done... skipping conversion...")
        return fname_out

    # sessions_in = sessions_in.replace(" ", "").split (",")

    # #
    # sess_list = []
    # for session in sessions_in:
    #     temp = session.replace("S",'')
    #     temp = 'MUnit_'+str(int(temp)-1)
    #     print ("session loaded: ", temp)
    #     sess_list.append(temp)
    #
    data = []
    with h5py.File(fname, 'r') as file:
                #
        for sess in sess_list:
            print ("processing: ", sess)
            temp = file['MSession_0'][sess]['Channel_0'][()]
            print ("    data loaded size: ", temp.shape)
            data.append(temp)

    data = np.vstack(data)
    print(data.shape)

    #
    tifffile.imwrite(fname_out, data)

    return fname_out
